In [75]:
import keras
import skimage

import numpy as np
import skvideo.io
import skimage.transform
import skimage.util
import csv
import collections
import os
from os import listdir
from keras.applications.vgg16 import VGG16
from keras.applications.densenet  import DenseNet121
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

from torchvision import transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
def normalize(image):
    '''
    normalize for pre-trained model input
    '''
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transform_input = transforms.Compose([
             transforms.ToPILImage(),
             transforms.Pad((0,40), fill=0, padding_mode='constant'),
             transforms.Resize(224),
             # transforms.CenterCrop(224),
    #         transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
#             normalize
        ])
    image_t = transform_input(image).numpy().transpose(1,2,0)
    # numpy output
    return image_t

def readShortVideo(video_path, video_category, video_name, downsample_factor=12, rescale_factor=1):
    '''
    @param video_path: video directory
    @param video_category: video category (see csv files)
    @param video_name: video name (unique, see csv files)
    @param downsample_factor: number of frames between each sampled frame (e.g., downsample_factor = 12 equals 2fps)
    @param rescale_factor: float of scale factor (rescale the image if you want to reduce computations)

    @return: (T, H, W, 3) ndarray, T indicates total sampled frames, H and W is heights and widths
    '''

    filepath = video_path + '/' + video_category
    filename = [file for file in os.listdir(filepath) if file.startswith(video_name)]
    video = os.path.join(filepath,filename[0])

    videogen = skvideo.io.vreader(video)
    frames = []
    
    
    for frameIdx, frame in enumerate(videogen):
        if frameIdx % downsample_factor == 0:
           # frame = skimage.transform.rescale(frame, rescale_factor, mode='constant', preserve_range=True).astype(np.uint8)
            frame_t = normalize(frame)
            frames.append(frame_t)
        else:
            continue

    return frames

In [77]:
frames = readShortVideo("HW5_data/TrimmedVideos/video/train/", 
                        "OP01-R01-PastaSalad", "OP01-R01-PastaSalad-1002316-1004005-F024051-F024101.mp4",
                        downsample_factor=12, rescale_factor=1)

In [78]:
np.stack(frames).shape

(5, 224, 224, 3)

In [79]:
all_video_path = []
all_video_frame = []
video_path = "HW5_data/TrimmedVideos/video/train/"
category_list = sorted(listdir(video_path))
for category in category_list:
    print(category)
    video_list_per_folder = sorted(listdir(os.path.join(video_path,category)))
    a = ["-".join(file_name.split("-")[:5]) for file_name in video_list_per_folder]
    all_video_path += a
    for video in video_list_per_folder:
        frames = readShortVideo(video_path, category, video, downsample_factor=12, rescale_factor=1)
        all_video_frame.append(np.stack(frames))

OP01-R01-PastaSalad
OP01-R04-ContinentalBreakfast
OP01-R05-Cheeseburger
OP01-R06-GreekSalad
OP01-R07-Pizza
OP02-R01-PastaSalad
OP02-R02-TurkeySandwich
OP02-R03-BaconAndEggs
OP02-R05-Cheeseburger
OP02-R06-GreekSalad
OP02-R07-Pizza
OP03-R01-PastaSalad
OP03-R03-BaconAndEggs
OP03-R04-ContinentalBreakfast
OP03-R05-Cheeseburger
OP03-R07-Pizza
OP04-R01-PastaSalad
OP04-R02-TurkeySandwich
OP04-R03-BaconAndEggs
OP04-R04-ContinentalBreakfast
OP04-R06-GreekSalad
OP04-R07-Pizza
OP05-R03-BaconAndEggs
OP05-R04-ContinentalBreakfast
OP06-R02-TurkeySandwich
OP06-R03-BaconAndEggs
OP06-R04-ContinentalBreakfast
OP06-R06-GreekSalad
OP06-R07-Pizza
